# Hyperparameter Tuning for SVM Linear Model

## Introduction

This notebook aims to optimize the performance of the Support vector machines linear model applied to the League of Legends matches dataset. We will compare different hyperparameter tuning methods: Grid Search, Randomized Search, and Bayesian Optimization.

## Installing Prerequisites


In [1]:
# !pip install catboost
# !pip install scikit-optimize
# !pip install category_encoders
# !pip install bayesian-optimization

## Importing Libraries

In [2]:
import time
import pickle
import numpy as np
import pandas as pd
from sklearn.svm import SVC
import category_encoders as ce
import matplotlib.pyplot as plt
from skopt import BayesSearchCV
from catboost import CatBoostClassifier
from bayes_opt import BayesianOptimization
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, roc_auc_score

## Loading Data

First, we load the dataset and split it into a training and testing sets.

In [3]:
#Loading Dataset
file_path = '../../sentiment_scores.csv'
df = pd.read_csv(file_path)

#Splitting into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

## Data Preprocessing

We preprocess the data using One-Hot Encoding and Target Encoding.

One-Hot Encoding the low cardinality variables, namely, 'League',  'Season' and 'Type'.

In [4]:
# One-Hot Encoding
train_df_onehot = pd.get_dummies(train_df, columns=['League', 'Season', 'Type'])
test_df_onehot = pd.get_dummies(test_df, columns=['League', 'Season', 'Type'])
missing_cols = set(train_df_onehot.columns) - set(test_df_onehot.columns)

# Handling missing columns
for c in missing_cols:
    test_df_onehot[c] = 0
test_df_onehot = test_df_onehot[train_df_onehot.columns]

Applying Target Encoding to the high cardinality variables.

In [5]:
# Target Encoding
target_cols = [
    'blueTop', 'blueJungle', 'blueMiddle', 'blueADC', 'blueSupport',
    'redTop', 'redJungle', 'redMiddle', 'redADC', 'redSupport',
    'blueTopChamp', 'blueJungleChamp', 'blueMiddleChamp', 'blueADCChamp', 'blueSupportChamp',
    'redTopChamp', 'redJungleChamp', 'redMiddleChamp', 'redADCChamp', 'redSupportChamp',
    'blueTeamTag', 'redTeamTag'
]
target_variable = 'bResult'
encoder = ce.TargetEncoder(cols=target_cols)
encoder.fit(train_df, train_df[target_variable])
train_df_target_encoded = encoder.transform(train_df)
test_df_target_encoded = encoder.transform(test_df)

Combining the One-Hot and Target Encoded Dataframes for training and testing sets.

In [6]:
# Removing Target Encoded Columns from One-Hot Encoded DataFrame
train_df_onehot = train_df_onehot.drop(columns=target_cols, axis=1)
test_df_onehot = test_df_onehot.drop(columns=target_cols, axis=1)

# Concatenating One-Hot and Target Encoded DataFrames:
train_df_encoded = pd.concat([train_df_onehot, train_df_target_encoded[target_cols]], axis=1)
test_df_encoded = pd.concat([test_df_onehot, test_df_target_encoded[target_cols]], axis=1)

## Feature Selection

We perform feature selection using averaged importance scores from mutual information and CatBoost.

In [7]:
# Separate features and target variable
X_train = train_df_encoded.drop([target_variable], axis=1)
y_train = train_df_encoded[target_variable]

# Calculate Mutual Information scores
mi_scores = mutual_info_classif(X_train, y_train)
mi_scores = pd.Series(mi_scores, name='MI_Scores', index=X_train.columns)

# CatBoost Importance
catboost_model = CatBoostClassifier(iterations=100, verbose=0)
catboost_model.fit(X_train, y_train)
catboost_importances = pd.Series(catboost_model.get_feature_importance(), name='CatBoost_Importance', index=X_train.columns)

# Combine and Normalize
importance_df = pd.concat([mi_scores, catboost_importances], axis=1)
importance_df['MI_Scores'] = (importance_df['MI_Scores'] - importance_df['MI_Scores'].min()) / (importance_df['MI_Scores'].max() - importance_df['MI_Scores'].min())
importance_df['CatBoost_Importance'] = (importance_df['CatBoost_Importance'] - importance_df['CatBoost_Importance'].min()) / (importance_df['CatBoost_Importance'].max() - importance_df['CatBoost_Importance'].min())
importance_df['Combined_Importance'] = (importance_df['MI_Scores'] + importance_df['CatBoost_Importance']) / 2

# Sort and Select Features
sorted_features = importance_df.sort_values(by='Combined_Importance', ascending=False).index
N = 22
selected_features = sorted_features[:N]
X_train_selected = X_train[selected_features]
X_test_selected = test_df_encoded.drop([target_variable], axis=1)[selected_features]

## Hyperparameter Tuning

We use three methods for hyperparameter tuning: Grid Search, Randomized Search, and Bayesian Optimization.

In [8]:
# Training the model to prepare for Hyperparameter Tuning
def train_evaluate_svm(params):
    svm_clf = SVC(
        C=params['C'],
        kernel='linear',
        random_state=42
    )
    svm_clf.fit(X_train_selected, y_train)
    y_pred = svm_clf.predict(X_test_selected)
    return accuracy_score(y_test, y_pred)

# Grid Search
param_grid = {
    'C': [0.1, 1, 10, 100]
}

In [9]:
# Grid Search
param_grid = {
    'C': [0.1, 1, 10, 100]
}

grid_search = GridSearchCV(SVC(kernel='linear', random_state=42), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_selected, y_train)
grid_best_params = grid_search.best_params_
grid_best_accuracy = grid_search.best_score_

In [10]:
# Randomized Search
param_dist = {
    'C': [0.1, 1, 10, 100]
}

random_search = RandomizedSearchCV(SVC(kernel='linear', random_state=42), param_dist, n_iter=4, cv=5, scoring='accuracy')
random_search.fit(X_train_selected, y_train)
random_best_params = random_search.best_params_
random_best_accuracy = random_search.best_score_

In [11]:
# Bayesian Optimization
y_test = test_df_encoded[target_variable]


def svm_bayesian(C):
    params = {
        'C': C
    }
    return train_evaluate_svm(params)

optimizer = BayesianOptimization(
    f=svm_bayesian,
    pbounds={
        'C': (0.1, 100)
    },
    random_state=42
)

optimizer.maximize(init_points=5, n_iter=15)
bayesian_best_params = optimizer.max['params']
bayesian_best_accuracy = optimizer.max['target']

|   iter    |  target   |     C     |
-------------------------------------
| 1         | 0.6438    | 37.52     |
| 2         | 0.6451    | 95.08     |
| 3         | 0.6451    | 73.23     |
| 4         | 0.6451    | 59.91     |
| 5         | 0.6478    | 15.69     |
| 6         | 0.6464    | 12.34     |
| 7         | 0.6478    | 19.1      |
| 8         | 0.6385    | 0.1       |
| 9         | 0.6438    | 84.25     |
| 10        | 0.6464    | 49.62     |
| 11        | 0.6464    | 26.44     |
| 12        | 0.6438    | 100.0     |
| 13        | 0.6451    | 53.91     |
| 14        | 0.6451    | 66.75     |
| 15        | 0.6478    | 22.27     |
| 16        | 0.6451    | 44.99     |
| 17        | 0.6478    | 17.24     |
| 18        | 0.6478    | 20.87     |
| 19        | 0.6478    | 16.37     |
| 20        | 0.6478    | 21.64     |


## Evaluating Models

After hyperparameter tuning, we evaluate the best models from each method on the test data.

In [12]:
def train_evaluate(params, X_train, y_train, X_test, y_test, method):
    if 'C' in params:
        params['C'] = float(params['C'])

    # Training the model and recording training time
    svm_clf = SVC(kernel='linear', random_state=42, **params)
    start_train_time = time.time()
    svm_clf.fit(X_train, y_train)
    end_train_time = time.time()

    # Testing the model and recording prediction time
    start_test_time = time.time()
    y_pred = svm_clf.predict(X_test)
    end_test_time = time.time()

    # Calculating the metrics
    test_accuracy = accuracy_score(y_test, y_pred)
    test_precision = precision_score(y_test, y_pred)
    test_recall = recall_score(y_test, y_pred)
    test_f1 = f1_score(y_test, y_pred)
    train_time = end_train_time - start_train_time
    test_time = end_test_time - start_test_time

    # Creating the results dataframe
    results_df = pd.DataFrame({
        'Method': [method],
        'Accuracy': [test_accuracy],
        'Precision': [test_precision],
        'Recall': [test_recall],
        'F1-Score': [test_f1],
        'Training Time (s)': [train_time],
        'Prediction Time (s)': [test_time]
    })
    return results_df

In [13]:
# Prepare the test data
X_test_selected = test_df_encoded.drop([target_variable], axis=1)[selected_features]
y_test = test_df_encoded[target_variable]

# Convert Bayesian best parameters to the correct format for SVM
bayesian_best_params = {key: float(value) if key == 'C' else value for key, value in bayesian_best_params.items()}

# Evaluate the model using best parameters from each method for SVM
results_grid_svm = train_evaluate(grid_best_params, X_train_selected, y_train, X_test_selected, y_test, 'Grid Search')
results_random_svm = train_evaluate(random_best_params, X_train_selected, y_train, X_test_selected, y_test, 'Random Search')
results_bayesian_svm = train_evaluate(bayesian_best_params, X_train_selected, y_train, X_test_selected, y_test, 'Bayesian Optimization')

## Results Comparison

Comparing the performance of the models using the best hyperparameters from Grid Search, Randomized Search, and Bayesian Optimization methods.

In [14]:
# Concatenate the results
final_results_svm = pd.concat([results_grid_svm, results_random_svm, results_bayesian_svm], axis=0).reset_index(drop=True)

# Display Best Parameters and Final Results
print(f"Grid Search - Best Params: {grid_best_params}, Best Accuracy: {grid_best_accuracy}")
print(f"Random Search - Best Params: {random_best_params}, Best Accuracy: {random_best_accuracy}")
print(f"Bayesian Optimization - Best Params: {bayesian_best_params}, Best Accuracy: {bayesian_best_accuracy}")

# Display the final results for SVM
print(final_results_svm)

Grid Search - Best Params: {'C': 100.0}, Best Accuracy: 0.717823896533574
Random Search - Best Params: {'C': 100.0}, Best Accuracy: 0.717823896533574
Bayesian Optimization - Best Params: {'C': 15.686262180199408}, Best Accuracy: 0.6477572559366754
                  Method  Accuracy  Precision    Recall  F1-Score  \
0            Grid Search  0.643799   0.663755  0.723810  0.692483   
1          Random Search  0.643799   0.663755  0.723810  0.692483   
2  Bayesian Optimization  0.647757   0.666667  0.728571  0.696246   

   Training Time (s)  Prediction Time (s)  
0           4.183389             0.140504  
1           4.233829             0.170529  
2           2.691170             0.169991  
